In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, ReLU, Flatten, Dense, Softmax
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import Sequence
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
df = open("?????????????????????").read().splitlines()
games = [i.split(',',2)[-1] for i in df]

In [18]:
chars = 'abcdefghijklmnopqrs'
coordinates = {k:v for v,k in enumerate(chars)}
chartonumbers = {k:v for k,v in enumerate(chars)}

In [19]:
def prepare_input(moves):
    x = np.zeros((19,19,4))
    for move in moves:
        color = move[0]
        column = coordinates[move[2]]
        row = coordinates[move[3]]
        if color == 'B':
            x[row,column,0] = 1
            x[row,column,2] = 1
        if color == 'W':
            x[row,column,1] = 1
            x[row,column,2] = 1
    if moves:
        last_move_column = coordinates[moves[-1][2]]
        last_move_row = coordinates[moves[-1][3]]
        x[row,column,3] = 1
    x[:,:,2] = np.where(x[:,:,2] == 0, 1, 0)
    return x

def prepare_label(move):
    column = coordinates[move[2]]
    row = coordinates[move[3]]
    return column*19+row

In [20]:
n_games = 0
n_moves = 0
for game in games:
    n_games += 1
    moves_list = game.split(',')
    for move in moves_list:
        n_moves += 1

In [ ]:
def data_generator(games, batch_size):
    def generator():
        x_batch = [] # Initialize data batch
        y_batch = [] # Initialize target batch
        for game in games: # Iterate through games
            moves_list = game.split(',')
            for count, move in enumerate(moves_list):
                x_batch.append(prepare_input(moves_list[:count]))
                y_batch.append(prepare_label(moves_list[count]))
                if len(x_batch) == batch_size: # Yield when reached batch size
                    yield np.array(x_batch), tf.one_hot(np.array(y_batch), depth=19*19)
                    x_batch = []
                    y_batch = []
    return generator

batch_size = 128
generator = data_generator(games, batch_size)
dataset = tf.data.Dataset.from_generator(generator, 
                                         output_types=(tf.float32, tf.float32),
                                         output_shapes=(tf.TensorShape((batch_size,19,19,4)),tf.TensorShape((batch_size,361)))
                                        )
dataset = dataset.prefetch(tf.data.AUTOTUNE)